In [2]:
import NN
import utils
import config as cfg

### Load Dataframes

In [ ]:
traindf = utils.load_file("[PATH]")
validdf = utils.load_file("[PATH]")
testdf  = utils.load_file("[PATH]")

### Setup Parameters

In [ ]:
ETF = 'SPY' # 'SPY', 'DIA', 'QQQ'
NNtype = 'MLP' # 'MLP', 'RNN', 'PSN'
params = cfg.train_parameters[ETF][NNtype]

### Initialize DataLoaders

In [ ]:
features_col = []
for i in range(params['input_size']):
    features_col.append("X_"_str(i+1))
target_col   = 'Target'

mu, sigma = None, None

trainloader = utils.DataFrame2DataLoader(traindf, features_col, target_col, batch_size=8, normalize=False)
validloader = utils.DataFrame2DataLoader(validdf, features_col, target_col, batch_size=4, normalize=False)
testloader  = utils.DataFrame2DataLoader(testdf, features_col, target_col, batch_size=4, normalize=False)

### Instanciate Model

In [ ]:
model = NN.Model(NNtype)
model.setup(params["input_size"], 
            params["hidden_size"],
            params["output_size"],
            params["epochs"],
            params["optim"],
            params["lr"],
            params["momentum"]
           )

### Training

In [ ]:
model.train(trainloader, validloader)

### Evaluation

In [ ]:
model.evaluate(testloader)